In [7]:
import pandas as pd
import numpy as np

In [8]:
#Reading in required files
df_comments = pd.read_csv ('user_commment_subreddits.csv', header = None)
df_posts = pd.read_csv ('user_posts_subreddits.csv', header = None)
df_stats = pd.read_csv ('user_stats.csv')

In [9]:
#flatten to list to get mode and remove NaN
#post_users = list(df_posts.iloc[:,0])
post_users = list(set(df_posts.iloc[:,0]) &set(df_stats["user_name"]))
df_posts = df_posts[df_posts.iloc[:,0].isin(post_users)].drop_duplicates(subset = 0)
del df_posts[df_posts.columns[0]]
posts_matrix = df_posts.to_numpy()
flat_posts = posts_matrix.flatten()
flat_posts = flat_posts[~pd.isnull(flat_posts)]
#post_matrix = post_matrix[~pd.isnull(post_matrix)]


comment_users = list(set(df_comments.iloc[:,0]) &set(df_stats["user_name"]))
df_comments = df_comments[df_comments.iloc[:,0].isin(comment_users)].drop_duplicates(subset = 0)
comments_matrix = df_comments.to_numpy()
del df_comments[df_comments.columns[0]]
flat_comments = comments_matrix.flatten()
flat_comments = flat_comments[~pd.isnull(flat_comments)]
#comment_matrix = comment_matrix[~pd.isnull(comment_matrix)]

In [10]:
#len(comment_users)
posts_matrix.shape
#comment_matrix.shape

(7166, 1003)

In [11]:
vals,counts = np.unique(flat_posts, return_counts=True)
index = np.argpartition(counts, -10)[-10:]
vals[index]

array(['unpopularopinion', 'lostgeneration', 'PoliticalHumor', 'memes',
       'politics', 'NoStupidQuestions', 'LateStageCapitalism',
       'AskReddit', 'Showerthoughts', 'antiwork'], dtype=object)

In [12]:
df_posts.count()

1       6820
2       6569
3       6387
4       6191
5       6020
        ... 
999       39
1000      26
1001       5
1002       2
1003       1
Length: 1003, dtype: int64

In [13]:
vals,counts = np.unique(flat_comments, return_counts=True)
index = np.argpartition(counts, -10)[-10:]
vals[index]

array(['PublicFreakout', 'facepalm', 'politics', 'WorkReform',
       'AmItheAsshole', 'AskReddit', 'news', 'antiwork',
       'WhitePeopleTwitter', 'worldnews'], dtype=object)

In [14]:
#Predictive model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [15]:
#df_stats = df_stats.drop_duplicates(subset = "user_name")
df_stats_posts = df_stats[df_stats['user_name'].isin(post_users)].drop_duplicates(subset = "user_name")
df_stats_comments = df_stats[df_stats['user_name'].isin(comment_users)].drop_duplicates(subset = "user_name")
ground_truth_post = df_stats_posts["post_karma"].to_numpy()
ground_truth_comment = df_stats_comments["post_karma"].to_numpy()

In [16]:
df_stats_posts.shape
df_stats_comments.shape

(6403, 8)

In [17]:
#
pnum_aw = []
pnum_ar = []
pnum_st = []
pnum_lsc = []
pnum_nsq = []
pnum_p = []
pnum_m = []
pnum_ph = []
pnum_lg = []
pnum_upo = []

cnum_aw = []
cnum_ar = []
cnum_wn = []
cnum_p = []
cnum_aita = []
cnum_wpt = []
cnum_n = []
cnum_fp = []
cnum_lg = []
cnum_upo = []


for row in posts_matrix:
    pnum_aw.append(np.count_nonzero(row == "antiwork"))
    pnum_ar.append(np.count_nonzero(row == "AskReddit"))
    pnum_st.append(np.count_nonzero(row == "Showerthoughts"))
    pnum_lsc.append(np.count_nonzero(row == "LateStageCapitalism"))
    pnum_nsq.append(np.count_nonzero(row == "NoStupidQuestions"))
    pnum_p.append(np.count_nonzero(row == "politics"))
    pnum_m.append(np.count_nonzero(row == "memes"))
    pnum_ph.append(np.count_nonzero(row == "PoliticalHumor"))
    pnum_lg.append(np.count_nonzero(row == "lostgeneration"))
    pnum_upo.append(np.count_nonzero(row == "unpopularopinion"))

for row in comments_matrix:
    cnum_aw.append(np.count_nonzero(row == "antiwork"))
    cnum_ar.append(np.count_nonzero(row == "AskReddit"))
    cnum_wn.append(np.count_nonzero(row == "worldnews"))
    cnum_p.append(np.count_nonzero(row == "politics"))
    cnum_aita.append(np.count_nonzero(row == "AmItheAsshole"))
    cnum_wpt.append(np.count_nonzero(row == "WhitePeopleTwitter"))
    cnum_n.append(np.count_nonzero(row == "news"))
    cnum_fp.append(np.count_nonzero(row == "facepalm"))
    


In [18]:
pnum_aw = np.array(pnum_aw, ndmin=2)
pnum_ar = np.array(pnum_ar,ndmin=2)
pnum_st = np.array(pnum_st, ndmin=2)
pnum_lsc = np.array(pnum_lsc, ndmin=2)
pnum_nsq = np.array(pnum_nsq, ndmin=2)
pnum_p = np.array(pnum_p, ndmin=2)
pnum_m = np.array(pnum_m, ndmin=2)
pnum_ph = np.array(pnum_ph, ndmin=2)
pnum_lg = np.array(pnum_lg, ndmin=2)
pnum_upo = np.array(pnum_upo, ndmin=2)

cnum_aw = np.array(cnum_aw, ndmin=2)
cnum_ar = np.array(cnum_ar, ndmin=2)
cnum_wn = np.array(cnum_wn, ndmin=2)
cnum_p = np.array(cnum_p, ndmin=2)
cnum_aita = np.array(cnum_aita, ndmin=2)
cnum_wpt = np.array(cnum_wpt, ndmin=2)
cnum_n = np.array(cnum_n, ndmin=2)
cnum_fp = np.array(cnum_fp, ndmin=2)

In [19]:
#concatenate x_vals
X_post = np.concatenate((pnum_aw, pnum_ar, pnum_st, pnum_lsc, pnum_nsq, pnum_p, pnum_m,
                 pnum_ph, pnum_lg, pnum_upo), axis = 0).T
X_comment = np.concatenate((cnum_aw, cnum_ar, cnum_wn, cnum_p, cnum_aita, cnum_wpt, cnum_n, cnum_fp), axis = 0).T

In [20]:
#standardizing
scaler = StandardScaler()
Xs_post = scaler.fit_transform(X_post)
Xs_comment = scaler.fit_transform(X_comment)

In [21]:
#Post Regression
post_reg = LinearRegression().fit(Xs_post, ground_truth_post)

print(post_reg.score(Xs_post, ground_truth_post))
post_reg.coef_

0.237346862266701


array([5937.29328581,   37.13756363,  373.92586002,  611.68332914,
         94.05473076, -257.86299196,  194.70325117,  109.95520986,
        787.83078362,  -16.40749527])

In [22]:
#Comment Regression
comment_reg = LinearRegression().fit(Xs_comment, ground_truth_comment)
print(comment_reg.score(Xs_comment, ground_truth_comment))
comment_reg.coef_

0.0028627584593269306


array([ 477.3563337 ,   54.2479749 , -146.05067414, -212.45103127,
       -219.07955377,  -29.28471388, -246.55633835, -209.51881827])